In [ ]:
#!/usr/bin/env python3
# import the required modules
from __future__ import print_function
#from click import exceptions
import pandas as pd
import numpy as np
import csv
import datetime
import statistics
import itertools
import time

import tensorflow as tf
#CUDA_VISIBLE_DEVICES=""
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import keras
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adamax, SGD, Adam
from keras import initializers
from keras.metrics import *
from keras import regularizers

from sklearn.metrics import confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize,LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pause = False

# inference mode
ep = 0
csv_pre = 'data/auto_pre_handover.csv' # 5.5s without pause
csv_post = 'data/auto_post_handover.csv' # 27.5s without pause

In [ ]:
# reshape panda.DataFrame to Keras style: (batch_size, time_step, nb_features)
def reshape_data(data, n_prev):
    docX = []
    # add rows of 0s if there are too few rows for time step padding
    df0 = pd.DataFrame(0.0, index=np.arange(n_prev), columns=data.columns)
    if len(data) < n_prev:
        data = pd.concat([data,df0])
    # time step padding
    for i in range(len(data)):
        if i < (len(data)-n_prev):
            docX.append(data[i:i+n_prev])
        else: # the frames in the last window use the same context
            docX.append(data[(len(data)-n_prev):len(data)])
    alsX = np.array(docX)
    return alsX

In [ ]:
# one-hot encoding of the class labels
def one_hot(labels):
    labels_converted = []
    for label in labels:
        if label == 'LEFT':
            label_converted = [1,0,0]
        elif label == 'MIDDLE':
            label_converted = [0,1,0]
        elif label == 'RIGHT':
            label_converted = [0,0,1]
        labels_converted.append(label_converted)
    labels_converted = np.asarray(labels_converted)
    return labels_converted

In [ ]:
# construct data
def feature_read(df_file, feat_cols, label_cols):
    # read in data
    data = pd.read_csv(df_file, header=0)

    # creating feature set
    feat_cols_end = feat_cols + 100 # only use keypoints (x,y,z,conf) features
    x_all = data.iloc[:, feat_cols:feat_cols_end]

    # creating label arrays
    y_otp = one_hot(data[label_cols[0]]) # OTP with 3 classes

    return x_all, y_otp

In [ ]:
# reverse the one-hot encoded predictions to origianl action labels
def one_hot_rev(labels):
    labels_converted = []
    for label in labels:
        if label == 0:
            label_converted = 'LEFT'
        elif label == 1:
            label_converted = 'MIDDLE'
        elif label == 2:
            label_converted = 'RIGHT'
        labels_converted.append(label_converted)
    labels_converted = np.asarray(labels_converted)
    return labels_converted

# reverse the one-hot encoded predictions to True/False action labels
def binary_rev(labels):
    labels_converted = []
    for label in labels:
        if label == 0:
            label_converted = False
        elif label == 1:
            label_converted = True
        labels_converted.append(label_converted)
    labels_converted = np.asarray(labels_converted)
    return labels_converted

In [ ]:
# use saved model to continuously generate predictions of new data
def inference(classifier = 'OTP', time_step = 5):
    # load the saved model
    file_log = 'inf_data/' + classifier + '_log.txt'
    pred_f = 'inf_data/' + classifier + '_pred.csv'
    model_name = 'inf_data/' + classifier + '_model.h5'
    model = load_model(model_name)

    # load data to be inferred on
    inf_f = 'inf_data/PoseMonitor.csv'
    inf_cols = 0 # OAK-D outcome only containing pose estimates

    # read in data
    df_inf = pd.read_csv(inf_f, header=0)
    inf_cols_end = inf_cols + 100 # only use keypoints (x,y,z,conf) features
    x_all_tst = df_inf.iloc[:, inf_cols:inf_cols_end]

    # time step padding
    X_tst = reshape_data(x_all_tst, time_step)

    # reshape data for Conv2D
    X_tst = X_tst.reshape(X_tst.shape[0],X_tst.shape[1],X_tst.shape[2],1)

    # generate predictions

    y_pred = model.predict(X_tst, verbose=0) # probabilities
    y_pred_cat = np.zeros(y_pred.shape[0], dtype=int) # initialise array to store class labels

    # adjustable prediction thresholds
    if classifier == 'OTP':
        thresholds = [0.4, 0.5, 0.1]
    else:
        thresholds = [0.6, 0.4]

    # use custom probability threshold
    for i in range(y_pred.shape[0]):
        for j in range(y_pred.shape[1]):
            if y_pred[i,j] > thresholds[j]:
                y_pred_cat[i] = j
                break

    # find the class with highest probability
    # y_pred_cat = np.argmax(y_pred, axis=1) 

    # save predictions as a dataframe column
    #df_pred = pd.DataFrame(columns=['handover status','arm status','base status'])
    df_pred = pd.DataFrame(columns=['predicted'])

    if classifier == 'OTP':
        y_pred_cat_label = one_hot_rev(y_pred_cat) # switch from one-hot class labels to original lables
    else:
        y_pred_cat_label = binary_rev(y_pred_cat) # switch from binary one-hot class labels to original lables
    df_pred['predicted'] = pd.Series(y_pred_cat_label)
    # get majority class as segmet prediction
    c_majority = df_pred['predicted'].value_counts()[:1].index.tolist()
    l_majority = c_majority[0]
    # print(y_pred)
    # print(y_pred_cat)
    # print(y_pred_cat_label)
    print(f'Majority class predicted for this segment is: {l_majority}')

    # save predictions to a csv file
    #df_pred.to_csv(pred_f, index=False)

    return l_majority

In [ ]:
# publish name of csv to be replayed to ROS so auto.py (copy of replay_data.py) knows which files to use based on the ML inputs
def ReplayDataMsg(replay_file):
    # pub.publish(replay_file) # publish file name to topic
    print(f'Replaying example segment: {replay_file}')

In [ ]:
# use saved model to continuously generate predictions of new data
while not pause:
    # when to run the pre-handover segment
    timeout = time.time() + 600 # (10min timeout)
    while True:
        start_ep = inference(classifier = 'BASE', time_step = 5)
        if start_ep or (time.time() > timeout):
            print("Executing pre-handover segment\n")
            # execute the pre-handover segment
            ReplayDataMsg(csv_pre)
            # predict where to perform the handover when episode starts
            prediction_otp = inference(classifier = 'OTP', time_step = 5)
            csv_reach = 'data/auto_' + prediction_otp + '_OTP_reach.csv' # 5.0s without pause
            csv_tuck = 'data/auto_' + prediction_otp + '_OTP_tuck.csv' # 5.1s without pause
            break
        time.sleep(0.5)

    # when to run the participant handover reaching segment
    while True:
        start_OTP = inference(classifier = 'ARM_reach', time_step = 5)
        if start_OTP or time.time() > timeout:
            # update prediction of where to perform the handover at the start of object exchange if needed
            prediction_otp = inference(classifier = 'OTP', time_step = 5)
            csv_reach = 'data/auto_' + prediction_otp + '_OTP_reach.csv' # 5.0s without pause
            csv_tuck = 'data/auto_' + prediction_otp + '_OTP_tuck.csv' # 5.1s without pause
            # execute the participant handover reaching segment based on the predicted OTP
            ReplayDataMsg(csv_reach)
            break
        time.sleep(0.5)

    # when to run the participant handover tucking and the consequent post-handover segment
    while True:
        start_fin = inference(classifier = 'ARM_tuck', time_step = 5)
        if start_fin or time.time() > timeout:
            ReplayDataMsg(csv_tuck) # execute the participant handover tucking segment based on the predicted OTP
            break
        time.sleep(0.5)

    # execute the post-handover segment
    ReplayDataMsg(csv_post)

    time.sleep(0.5)

    episode = [csv_pre, csv_reach, csv_tuck, csv_post] # 43.1s without pause
    print(f'Finished episode No.{ep}, sequence: {episode}')
    ep = ep + 1 # episode number counter
